In [78]:
import pandas as pd
import numpy as np

In [79]:
def one_hot_list(data, column, prefix=None):
    data[column] = data[column].str.split(", ")
    data = data.explode(column)
    data_enc = pd.get_dummies(data[column], prefix=prefix)
    encoded_data = pd.concat([data, data_enc], axis=1)
    return encoded_data

# Courses compulsory for each degree

In [80]:
enrol_nums_data = pd.read_excel("Raw Data/Course Enrollment Numbers.xlsx", sheet_name="abbreviated course names")

In [81]:
enrol_nums_data.to_excel("Processed Data/web_scraped_data.xlsx", index = False)

In [82]:
enc_tt_data = pd.read_excel("Processed Data/encoded_timetable_data.xlsx")

enc_tt_data = enc_tt_data[enc_tt_data['enrollment_number'] > 0]

enc_tt = enc_tt_data.drop_duplicates(subset = ["Code"])

In [83]:

compulsory_enc = one_hot_list(enc_tt, "Degrees Compulsory For")
# compulsory_enc.to_excel("compulsory_encoded.xlsx", index = False)

<ipython-input-79-6d4a8d5a498f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data[column].str.split(", ")


In [84]:
# list(compulsory_enc.columns)[11:]

## Write the course codes from the above datasets into a list of lists to the txt file

In [85]:
comp_ambsc = compulsory_enc[compulsory_enc["ambsc"] == True] 
comp_ammmath = compulsory_enc[compulsory_enc["ammmath"]== True]
comp_cammsc = compulsory_enc[compulsory_enc["cammsc"]== True]
comp_cmfft = compulsory_enc[compulsory_enc["cmfft"]== True]
comp_cmfpt = compulsory_enc[compulsory_enc["cmfpt"]== True]
comp_fmoft = compulsory_enc[compulsory_enc["fmoft"]== True] 
comp_fmopt = compulsory_enc[compulsory_enc["fmopt"]== True]
comp_mbsc = compulsory_enc[compulsory_enc["mbsc"]== True]
comp_mma = compulsory_enc[compulsory_enc["mma"]== True]
comp_mmmath = compulsory_enc[compulsory_enc["mmmath"]== True]
comp_mnbi = compulsory_enc[compulsory_enc["mnbi"]== True] 
comp_mnbu = compulsory_enc[compulsory_enc["mnbu"]== True]
comp_mnmu = compulsory_enc[compulsory_enc["mnmu"]== True]  
comp_mnph = compulsory_enc[compulsory_enc["mnph"]== True] 
comp_mnst = compulsory_enc[compulsory_enc["mnst"]== True]
comp_orco = compulsory_enc[compulsory_enc["orco"] == True] 
comp_ords = compulsory_enc[compulsory_enc["ords"] == True]
comp_ormsc = compulsory_enc[compulsory_enc["ormsc"] == True]
comp_orwr = compulsory_enc[compulsory_enc["orwr"] == True]
comp_stds = compulsory_enc[compulsory_enc["stds"] == True]
comp_storft = compulsory_enc[compulsory_enc["storft"] == True]  
comp_storpt = compulsory_enc[compulsory_enc["storpt"] == True] 

# Courses within each category

In [86]:
category_enc = one_hot_list(enc_tt, "Categories")

<ipython-input-79-6d4a8d5a498f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data[column].str.split(", ")


In [87]:
categories = list(category_enc.columns)[11:]

In [88]:
algebra_courses = category_enc[category_enc["Algebra"] == True]
analysis_courses = category_enc[category_enc["Analysis"] == True]
app_comp_courses = category_enc[category_enc["Applied & Computational"] == True]
data_science_courses = category_enc[category_enc["Data Science"] == True]
financial_courses = category_enc[category_enc["Financial"] == True]
geom_top_courses = category_enc[category_enc["Geometry & Topology"] == True]
mat_phys_courses = category_enc[category_enc["Mathematical Physics"] == True]
opt_or_courses = category_enc[category_enc["Optimization & Operational Research"] == True]
probability_courses = category_enc[category_enc["Probability"] == True]
statistics_courses = category_enc[category_enc["Statistics"] == True]

# Testing

In [89]:
# enc_tt_grouped = enc_tt.sort_values(by=["ws_groups"])

# Sets of Rooms

In [90]:
room_data = pd.read_excel("Processed Data/encoded_room_data.xlsx")

In [91]:
all_rooms = room_data
lecture_theatres = room_data[room_data["layout_theatre"] == True]
classrooms = room_data[(room_data["layout_classroom"] | room_data["layout_boardroom"])]
comp_labs = room_data[room_data["layout_comp_lab"] == True]

pseudo_lec_theatres = ["JCMB_1501", "JCMB_5326", "JCMB_5327", "JCMB_5328", "MH_G.31", "40GS_13.07", "7-8CS1.01", "AT_2.05", "CMB_Seminar Room 5"]

## Capacities and Enrollment Numbers

# Writing to txt

In [92]:
proc_tt = pd.read_excel("Processed Data/encoded_timetable_data.xlsx").copy()
proc_tt.drop(proc_tt.index[proc_tt.Code == "MATH11220"].tolist(), inplace = True)
courses_unique = proc_tt.drop_duplicates(subset = ["Code"])
courses_unique.columns

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
periods = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 
           '10', '11', '12', '13', '14', '15', '16', '17', '18']

In [93]:
# test_grouping  = comp_ambsc.groupby('Normal Year Taken')['Code'].agg(list)

# test_grouping.to_dict()

In [94]:
def write_col(df, col, title, list_of_lists = False):
    if list_of_lists == False:
        output = f"{title}: [ " + " ".join('"' + value + '"' for value in list(df[col].astype(str).values)) + " ]\n\n"
    elif list_of_lists == True:
        output = "[ " + " ".join('"' + value + '"' for value in list(df[col].astype(str).values)) + " ]\n"
    return output

def write_lolol(df, col1, col2):
    grouped_lists = df.groupby(col2)[col1].apply(list)
    inner_lists = []
    for values_list in grouped_lists.values:
        inner_lists.append(f"[{' '.join(map(str, values_list))}]")
    output = f'[{"  ".join(inner_lists)}]\n'
    return output
    

def write_num_col(df, col, title):
    file.write(f"{title}: [")
    np.savetxt(file, df[col].values, fmt='%s', delimiter=' ', newline=" ")
    file.write("]\n\n")

def write_list(lst, title):
    output = f"{title}: [ " + " ".join('"' + value + '"' for value in lst) + " ]\n\n"
    return output

## Replacing P with 5 in the Normal Year Taken column

In [95]:
enc_tt.loc[(enc_tt["Normal Year Taken"] == "P"), "Normal Year Taken"] = 5

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [96]:
course_categories_dict = {
    "algebra_courses": algebra_courses,
    "analysis_courses": analysis_courses,
    "app_comp_courses": app_comp_courses,
    "data_science_courses": data_science_courses,
    "financial_courses": financial_courses,
    "geom_top_courses": geom_top_courses,
    "mat_phys_courses": mat_phys_courses,
    "opt_or_courses": opt_or_courses,
    "probability_courses": probability_courses,
    "statistics_courses": statistics_courses
}

course_compulsory_dict = {
    "comp_ambsc": comp_ambsc,
    "comp_ammmath": comp_ammmath,
    "comp_cammsc": comp_cammsc,
    "comp_cmfft": comp_cmfft,
    "comp_cmfpt": comp_cmfpt,
    "comp_fmoft": comp_fmoft,
    "comp_fmopt": comp_fmopt,
    "comp_mbsc": comp_mbsc,
    "comp_mma": comp_mma,
    "comp_mmmath": comp_mmmath,
    "comp_mnbi": comp_mnbi,
    "comp_mnbu": comp_mnbu,
    "comp_mnmu": comp_mnmu,
    "comp_mnph": comp_mnph,
    "comp_mnst": comp_mnst,
    "comp_orco": comp_orco,
    "comp_ords": comp_ords,
    "comp_ormsc": comp_ormsc,
    "comp_orwr": comp_orwr,
    "comp_stds": comp_stds,
    "comp_storft": comp_storft,
    "comp_storpt": comp_storpt
}

In [97]:
compulsory_enc.columns[-22:]

Index(['ambsc', 'ammmath', 'cammsc', 'cmfft', 'cmfpt', 'fmoft', 'fmopt',
       'mbsc', 'mma', 'mmmath', 'mnbi', 'mnbu', 'mnmu', 'mnph', 'mnst', 'orco',
       'ords', 'ormsc', 'orwr', 'stds', 'storft', 'storpt'],
      dtype='object')

In [98]:
category_enc.columns[-10:]

Index(['Algebra', 'Analysis', 'Applied & Computational', 'Data Science',
       'Financial', 'Geometry & Topology', 'Mathematical Physics',
       'Optimization & Operational Research', 'Probability', 'Statistics'],
      dtype='object')

## GROUPED

In [100]:
output_file = f"xpress_data.txt"

# Writing to the file
with open(output_file, 'w') as file:

    file.write(write_list(days, "days"))
    file.write(write_list(periods, "time_periods"))
    file.write(write_list(list(compulsory_enc.columns)[-22:], "DegreeProgrammes"))
    file.write(write_list(list(category_enc.columns)[-10:], "Categories"))
    file.write("\n\n")

    file.write(write_col(courses_unique, "Code", "CourseID"))
    file.write(write_col(courses_unique, "Abbreviated Course Name", "CourseName"))
    write_num_col(courses_unique, "enrollment_number", "EnrollmentNumber")
    file.write("\n\n")


    file.write("CategoryCourseGroupings: [\n")
    for category, category_df in course_categories_dict.items():
        file.write(write_lolol(category_df, "Code", "encoded_year_taken"))
    file.write("]")
    file.write("\n\n")

    file.write("CompulsoryCourseGroupings: [\n")
    for comp_degree, compulsory_df in course_compulsory_dict.items(): 
        file.write(write_lolol(compulsory_df, "Code", "Normal Year Taken"))
    file.write("]")
    file.write("\n\n\n")


    file.write(write_col(all_rooms, "room_name", "Rooms"))
    file.write(write_col(all_rooms, "room_layout", "RoomType"))
    write_num_col(all_rooms, "capacity", "RoomCapacities")
    file.write("\n\n")

    file.write(write_col(lecture_theatres, "room_name", "RoomsLecture"))
    file.write(write_list(pseudo_lec_theatres, "PseudoLecTheatres"))
    file.write(write_col(classrooms, "room_name", "RoomsClassroom"))
    file.write(write_col(comp_labs, "room_name", "RoomsCompWorkshop"))

In [66]:
# output_file = f"xpress_data.txt"

# # Writing to the file
# with open(output_file, 'w') as file:

#     file.write(write_list(days, "days"))
#     file.write(write_list(periods, "time_periods"))
#     file.write(write_list(list(compulsory_enc.columns)[-22:], "DegreeProgrammes"))
#     file.write(write_list(list(category_enc.columns)[-10:], "Categories"))
#     file.write("\n\n")

#     file.write(write_col(courses_unique, "Code", "CourseID"))
#     file.write(write_col(courses_unique, "Abbreviated Course Name", "CourseName"))
#     write_num_col(courses_unique, "enrollment_number", "EnrollmentNumber")
#     file.write("\n\n")


#     file.write("CategoryCourseGroupings: [\n")
#     for category, category_df in course_categories_dict.items():
#         file.write(write_lolol(category_df, "Code", "Normal Year Taken"))
#     file.write("]")
#     file.write("\n\n")

#     file.write("CompulsoryCourseGroupings: [\n")
#     for comp_degree, compulsory_df in course_compulsory_dict.items(): 
#         file.write(write_lolol(compulsory_df, "Code", "Normal Year Taken"))
#     file.write("]")
#     file.write("\n\n\n")


#     file.write(write_col(all_rooms, "room_name", "Rooms"))
#     file.write(write_col(all_rooms, "room_layout", "RoomType"))
#     write_num_col(all_rooms, "capacity", "RoomCapacities")
#     file.write("\n\n")

#     file.write(write_col(lecture_theatres, "room_name", "RoomsLecture"))
#     file.write(write_list(pseudo_lec_theatres, "PseudoLecTheatres"))
#     file.write(write_col(classrooms, "room_name", "RoomsClassroom"))
#     file.write(write_col(comp_labs, "room_name", "RoomsCompWorkshop"))

## UNGROUPED

In [31]:
output_file = f"xpress_test_data.txt"

# Writing to the file
with open(output_file, 'w') as file:

    file.write(write_list(days, "days"))
    file.write(write_list(periods, "time_periods"))
    file.write(write_list(list(compulsory_enc.columns)[-22:], "DegreeProgrammes"))
    file.write(write_list(list(category_enc.columns)[-10:], "Categories"))
    file.write("\n\n")

    file.write(write_col(courses_unique, "Code", "CourseID"))
    file.write(write_col(courses_unique, "Abbreviated Course Name", "CourseName"))
    write_num_col(courses_unique, "enrollment_number", "EnrollmentNumber")
    file.write("\n\n")



    file.write("CategoryCourseGroupings: [\n")
    for category, category_df in course_categories_dict.items():
        file.write(write_col(category_df, "Code", category, list_of_lists=True))
    file.write("]")
    file.write("\n\n")

    file.write("CompulsoryCourseGroupings: [\n")
    for comp_degree, compulsory_df in course_compulsory_dict.items(): 
        file.write(write_col(compulsory_df, "Code", comp_degree, list_of_lists=True))
    file.write("]")

    file.write("\n\n")

    file.write(write_col(all_rooms, "room_name", "Rooms"))
    file.write(write_col(all_rooms, "room_layout", "RoomType"))
    write_num_col(all_rooms, "capacity", "RoomCapacities")
    file.write("\n\n")

    file.write(write_col(lecture_theatres, "room_name", "RoomsLecture"))
    file.write(write_list(pseudo_lec_theatres, "PseudoLecTheatres"))
    file.write(write_col(classrooms, "room_name", "RoomsClassroom"))
    file.write(write_col(comp_labs, "room_name", "RoomsCompWorkshop"))